## 1. Libraries

In [1]:
import numpy as np
import pandas as pd 
import contractions
import emoji
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from textblob import Word
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from textblob import TextBlob
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

## 2. Dataset

In [2]:
data = pd.read_csv(os.path.join(os.getcwd(),"../static/dataset/emoji_dataset.csv"))
data

,Text,Emotion
0,im feeling groggy and horrid,sadness
1,i could feel the muscles in my arches ankles a...,joy
2,i feel like but im not very fond of that word 💙,love
3,i have to move stop staring at the other ladie...,joy
4,i have this kind of life so my girlfriend woul...,sadness
...,...,...
19995,im feeling ive resolved to live a life of love...,joy
19996,i used feel frustrated all the time,anger
19997,im starting to feel more sociable again i actu...,joy
19998,i am feeling devastated the inner voice within...,sadness


In [3]:
# Converting chat conversion words to normal words

def short_to_original(text):
	"""
	Return :- text after converting short_form words to original
	Input :- string
	Output :- string
	"""
	new_text = []
	for w in text.split():
		if w.upper() in chat_words_list:
			new_text.append(chat_words_map_dict[w.upper()])
		else:
			new_text.append(w)
	return " ".join(new_text)

chat_words_map_dict = {}
chat_words_list = []

# open short_form file and then read sentences from text file using read())
short_form_path = os.path.join(os.getcwd(),"../static/dataset/short_form.txt")
short_form_list = open(short_form_path, 'r')
chat_words_str = short_form_list.read()
lines = chat_words_str.split("\n")

for line in lines:
    try:
        cw, cw_expanded = line.split("\t",2)[:2]
        chat_words_list.append(cw.strip())
        chat_words_map_dict[cw.strip()] = cw_expanded.strip()
    except:
        pass
chat_words_list = set(chat_words_list)

In [4]:
data = data[['Text','Emotion']]

In [5]:
from sklearn.model_selection import train_test_split
train, validate, test = np.split(data.sample(frac=1, random_state=42), [int(.8*len(data)), int(0.9*len(data))])

In [6]:
train.shape

(16000, 2)

## 3. Preprocessing

In [7]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [23]:
def convert_emoji(text):
    return emoji.demojize(text)

def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word 

def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens=[convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

def expand_contractions(text):
    expanded_words = []   
    for word in text.split():
        expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

def convert_to_lower(text):
    words = text.split()
    for i in range(len(words)):
        words[i] = words[i].lower()
    sentence = " ".join(words)
    return sentence

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence

def remove_non_alphanum(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

def word_check(word):
    pattern = re.compile(r"(.)\1{2,}")
    sentence = pattern.sub(r"\1\1", word)
    b = TextBlob(sentence)
    return str(b.correct())

def remove_extendedwords(text):
    words = text.split()
    for i in range(len(words)):
        words[i] = word_check(words[i])
    sentence = " ".join(words)
    return sentence

def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    tag_map = {'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV, 'J': wordnet.ADJ}

    lemmatized_words = []
    for word, tag in pos_tags:
        wn_tag = tag_map.get(tag[0].upper(), wordnet.NOUN)
        if wn_tag == wordnet.VERB and word.endswith('ed'):
            wn_tag = wordnet.VERB
            word = word[:-2]
        lemmatized_word = lemmatizer.lemmatize(word, pos=wn_tag)
        lemmatized_words.append(lemmatized_word)

    # join the lemmatized words back into a sentence
    lemmatized_sentence = ' '.join(lemmatized_words)

    return (lemmatized_sentence)

In [24]:
train['Text'] = train['Text'].apply(lambda x: convert_emoji(x))
train['Text'] = train['Text'].apply(lambda x: convert_abbrev_in_text(x))
train['Text'] = train['Text'].apply(lambda x: expand_contractions(x))
train['Text'] = train['Text'].apply(lambda x: convert_to_lower(x))
train['Text'] = train['Text'].apply(lambda x: remove_stopwords(x))
train['Text'] = train['Text'].apply(lambda x: remove_non_alphanum(x))
train['Text'] = train['Text'].apply(lambda x: lemmatize_text(x))
train.Text

KeyboardInterrupt: 

In [ ]:
test['Text'] = test['Text'].apply(lambda x: convert_emoji(x))
test['Text'] = test['Text'].apply(lambda x: convert_abbrev_in_text(x))
test['Text'] = test['Text'].apply(lambda x: expand_contractions(x))
test['Text'] = test['Text'].apply(lambda x: convert_to_lower(x))
test['Text'] = test['Text'].apply(lambda x: remove_stopwords(x))
test['Text'] = test['Text'].apply(lambda x: remove_non_alphanum(x))
test['Text'] = test['Text'].apply(lambda x: lemmatize_text(x))
test.Text

In [11]:
validate['Text'] = validate['Text'].apply(lambda x: convert_emoji(x))
validate['Text'] = validate['Text'].apply(lambda x: convert_abbrev_in_text(x))
validate['Text'] = validate['Text'].apply(lambda x: expand_contractions(x))
validate['Text'] = validate['Text'].apply(lambda x: convert_to_lower(x))
validate['Text'] = validate['Text'].apply(lambda x: remove_stopwords(x))
validate['Text'] = validate['Text'].apply(lambda x: remove_non_alphanum(x))
validate['Text'] = validate['Text'].apply(lambda x: lemmatize_text(x))
validate.Text

1262                              feel offend think justly
19010    spend two week zombie mode two week feel feeli...
7212     love idea white blouse jumper feel jumper woul...
975               could help feel infuriated left building
2566     think notice prone feel jealous right help sho...
                               ...                        
10900                            angry feeling disillusion
7758     feel like someone need invest money could gorg...
4837     id let kill matter fact feel frightfully well ...
6548     feel though people find quite pleasant smiling...
4481     even think would ready fuck buddys emotion wou...
Name: Text, Length: 2000, dtype: object

## 4. Model

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_texts = train['Text']
tokenizer = Tokenizer(15212,lower=True,oov_token='UNK')
tokenizer.fit_on_texts(train_texts)

print('Found %d unique words.' % len(tokenizer.word_index))

# texts_to_sequences: Transforms each text in texts to a sequence of integers. 
# It basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.

train_texts_sequences = tokenizer.texts_to_sequences(train_texts)

# pad_sequences: Ensure that all sequences in a list have the same length. 
train_texts_pad_sequences = pad_sequences(train_texts_sequences, maxlen=80, padding='post') 

Found 15232 unique words.


In [13]:
from tensorflow.keras.utils import to_categorical
emotions = {'sadness': 0, 'joy': 1, 'surprise': 2, 'love': 3, 'anger': 4, 'fear': 5}

# Step 1: Replace all emotion values with integers
train['Emotion'] = train.Emotion.replace(emotions)
train_emotion_integers = train['Emotion'].values

# Step 2: Changing the integers to binary
train_emotion_categorical = to_categorical(train_emotion_integers)
train_emotion_categorical[:6] 

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [14]:
validate_texts = validate['Text']
validate_emotion_integers = validate.Emotion.replace(emotions)
validate_texts_sequences = tokenizer.texts_to_sequences(validate_texts)
validate_texts_pad_sequences = pad_sequences(validate_texts_sequences, maxlen=80, padding='post')
validate_emotion_categorical = to_categorical(validate_emotion_integers.values)
validate_emotion_categorical[:6]

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [15]:
import tensorflow as tf
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  print("All devices: ", tf.config.list_logical_devices('TPU'))
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
  tpu_strategy = tf.distribute.get_strategy() 

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Embedding,Dropout

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model=Sequential()
    model.add(Embedding(15212,64,input_length=80))
    model.add(Dropout(0.6))
    model.add(Bidirectional(LSTM(80,return_sequences=True)))
    model.add(Bidirectional(LSTM(160)))
    model.add(Dense(len(emotions),activation='softmax'))
    print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 64)            973568    
                                                                 
 dropout (Dropout)           (None, 80, 64)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 80, 160)          92800     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 320)              410880    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 1926      
                                                                 
Total params: 1,479,174
Trainable params: 1,479,174
Non-

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
hist=model.fit(train_texts_pad_sequences, train_emotion_categorical, epochs=10, validation_data = (validate_texts_pad_sequences, validate_emotion_categorical))

Epoch 1/10
500/500 [==============================] - 267s 509ms/step - loss: 1.1356 - accuracy: 0.5693 - val_loss: 0.5642 - val_accuracy: 0.8130
Epoch 2/10
500/500 [==============================] - 249s 499ms/step - loss: 0.4399 - accuracy: 0.8514 - val_loss: 0.3335 - val_accuracy: 0.8890
Epoch 3/10
500/500 [==============================] - 287s 575ms/step - loss: 0.2422 - accuracy: 0.9179 - val_loss: 0.2535 - val_accuracy: 0.9115
Epoch 4/10
500/500 [==============================] - 275s 550ms/step - loss: 0.1712 - accuracy: 0.9408 - val_loss: 0.2258 - val_accuracy: 0.9215
Epoch 5/10
500/500 [==============================] - 284s 568ms/step - loss: 0.1376 - accuracy: 0.9509 - val_loss: 0.2079 - val_accuracy: 0.9210
Epoch 6/10
500/500 [==============================] - 259s 518ms/step - loss: 0.1159 - accuracy: 0.9598 - val_loss: 0.2142 - val_accuracy: 0.9250
Epoch 7/10
500/500 [==============================] - 261s 522ms/step - loss: 0.1004 - accuracy: 0.9634 - val_loss: 0.2306 -

In [19]:
test_texts = test['Text']
test_emotion_integers = test.Emotion.replace(emotions)
test_texts_sequences = tokenizer.texts_to_sequences(test_texts)
test_texts_pad_sequences = pad_sequences(test_texts_sequences, maxlen=80, padding='post')
test_emotion_categorical = to_categorical(test_emotion_integers.values)
test_emotion_categorical[:7]

array([[0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [20]:
x = model.evaluate(test_texts_pad_sequences, test_emotion_categorical)
print(x)

63/63 [==============================] - 10s 162ms/step - loss: 0.2987 - accuracy: 0.9140
[0.29871219396591187, 0.9139999747276306]


In [21]:
import pickle
# dump information to that file
pickle.dump(tokenizer, open('../static/model/tokenizer.pkl', 'wb'))

# model.save("../static/model/final_model.hdf5")

In [27]:
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
import numpy
import os
model_json = model.to_json()
with open("../static/model/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../static/model/model.h5")
print("Saved model to disk")

Saved model to disk


In [28]:
import pickle

emotions = {'sadness': 0, 'joy': 1, 'surprise': 2, 'love': 3, 'anger': 4, 'fear': 5}

tokenizer_file = open("../static/model/tokenizer.pkl","rb")
tokenizer = pickle.load(tokenizer_file)
tokenizer_file.close()

json_file = open('../static/model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("../static/model/model.h5")

In [29]:
def get_key(value):
    for key,val in emotions.items():
          if (val==value):
            return key
        
def predict(sentence):
    sentence = convert_emoji(sentence)
    sentence = convert_abbrev_in_text(sentence)
    sentence = expand_contractions(sentence)
    sentence = chat_words_conversion(sentence)
    sentence = convert_to_lower(sentence)
    sentence = remove_stopwords(sentence)
    sentence = remove_non_alphanum(sentence)
    sentence = remove_extendedwords(sentence)
    sentence = lemmatize_text(sentence)
    print(sentence)
    sentence_lst=[]
    sentence_lst.append(sentence)
    sentence_seq=tokenizer.texts_to_sequences(sentence_lst)
    sentence_padded=pad_sequences(sentence_seq,maxlen=80,padding='post')
    certaintyprediction = model.predict(sentence_padded)[0]
    for key,val in emotions.items():
          print(key + ': ' + str(round(certaintyprediction[val]*100, 2)) + ' %')
    bestpredictionindex = np.argmax(certaintyprediction)
    certainty = str(round(certaintyprediction[bestpredictionindex]*100, 2))
    print('\nI am '+ certainty + ' % sure the emotion is ' + get_key(bestpredictionindex) + '.')

## 5. Testing

In [30]:
sentence = "🎁"
predict(sentence)

wrapped_gift
1/1 [==============================] - 4s 4s/step
sadness: 1.89 %
joy: 7.0 %
surprise: 57.26 %
love: 2.86 %
anger: 2.81 %
fear: 28.18 %

I am 57.26 % sure the emotion is surprise.


In [ ]:
predict("this is niceee😁")

In [ ]:
sentence = "A wedding can be a highly emotional event."
predict(sentence)

In [ ]:
sentence = "You are being very rude."
predict(sentence)

In [ ]:
predict("Today I went to Bangalore railway station, Yeshwantpura, to receive my uncle and aunt who were coming from Mumbai. It was a bright sunny day. Sun was shining like a star. While I and my father were crossing the Orion mall, we saw three elephants that made me reminded of my Kerala trip.Last year I went on a Kerala trip, where we visited around 5 cities like Cochin, Wayanad, Munnar, Kovalam, and Alappuzha. All the places were really awesome and beautiful. Then we went to Elephant junction Thekkady, Kumily, where people go for elephant rides. I rode sitting above the elephant around for 2 and half hours. Then we have also done elephant bath and feeding. We took a lot of pictures with elephants. It was a nice trip and I still can’t get over it.")